In [ ]:
!pip install ipynb
!pip install import-ipynb

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/MTApplication/src/

/content/drive/MyDrive/MTApplication/src


In [ ]:
import import_ipynb
import os
import pandas as pd
import Navigator
import Scraper
import FileHandler
import Preprocessor
import Translator

importing Jupyter notebook from FileHandler.ipynb
importing Jupyter notebook from Preprocessor.ipynb


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Examples of browser navigation and scraping

In [ ]:
# Example of clicking language checkbox on video site
navigator = Navigator.Navigator()
navigator.navigate("https://ailt.ilrdf.org.tw/colloquial/index")
navigator.click_button(Navigator.By.XPATH, f"//div[div[label[text()='{Navigator.Language.AMIS.value[0]}']]]")

In [ ]:
# Example of exhausting "load more" button clicks
navigator.click_button_repeatedly(Navigator.By.ID, "btnMoreCollection")

html = navigator.get_source_html()

In [ ]:
scraper = Scraper.getScraper(Scraper.ScrapeType.NTU)

In [ ]:
scraper.set_doc_locations("/content/drive/MyDrive/MTApplication/data/raw/NTU_Formosan/Amis/Amis_Nr-pear_zuomei.txt",
                          "/content/drive/MyDrive/MTApplication/data/raw/NTU_Formosan/Amis/Amis_Nr-pear_zuomei.en")

In [ ]:
matrix = scraper.parallel_scrape()

In [ ]:
print(matrix[:5])

Data preprocessing


In [ ]:
fileHandler = FileHandler.FileHandler()
raw_corpus = fileHandler.read_csv('/content/drive/MyDrive/MTApplication/data/pre-processed/amis-english-expanded.csv')

In [ ]:
ami_context = Preprocessor.Context("Amis", raw_corpus)
ami_context.set_strategy([
    Preprocessor.NormalizePunct(),
    Preprocessor.Truecase(),
    Preprocessor.RemoveParenthetical(),
    Preprocessor.RemovePunct(),
    Preprocessor.NormalizeSpaces(),
    Preprocessor.ReplaceSeq('o', 'u')
])

In [ ]:
eng_context = Preprocessor.Context("English", raw_corpus)
eng_context.set_strategy([
    Preprocessor.NormalizePunct(),
    Preprocessor.ReplaceSeq('&quot', ''),
    Preprocessor.PurgeNonEnglish(),
    Preprocessor.ReplaceSeq("sth.", "something"),
    Preprocessor.ReplaceSeq("sb.", "somebody"),
    Preprocessor.ReplaceSeq("-", " "),
    Preprocessor.ExpandContractions(),
    Preprocessor.Truecase(),
    Preprocessor.Dedupe(),
    Preprocessor.RemoveParenthetical(),
    Preprocessor.HandleOneToMany(),
    Preprocessor.RemovePunct(),
    Preprocessor.NormalizeSpaces(),
    Preprocessor.Spellcheck()
])

In [ ]:
preprocessor = Preprocessor.Preprocessor(ami_context, eng_context)

In [ ]:
processed_corpus = preprocessor.process((0, preprocessor.get_total_lines()))

In [ ]:
csv_formatted = [{"Amis": lst[0], "English": lst[1]} for lst in processed_corpus]
fileHandler.write_csv(
    ["Amis", "English"],
    csv_formatted,
    f'/content/drive/MyDrive/MTApplication/data/processed/amis-english-processed.csv'
  )

Model training

In [ ]:
model = Translator.Translator()
df = pd.read_csv('/content/drive/MyDrive/MTApplication/data/processed/amis-english-processed.csv')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 256205. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


In [ ]:
model.train(df, save_to='/content/drive/MyDrive/MTApplication/models/nllb-eng-ami-v1')

  0%|          | 0/50000 [00:00<?, ?it/s]

Step 0, Loss: 5.444636344909668


KeyboardInterrupt: 